## Getting CT acquisition data for preoperative and postoperative datasets from the DICOM metadata

In [ ]:
import os
import pydicom
import pandas as pd
import numpy as np
from datetime import datetime
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom
from scipy.stats import mannwhitneyu

# === Input paths ===
nii_folder = r"------ INSERT PATH HERE ------"
dicom_root = r"------ INSERT PATH HERE ------"
series_excel = r"------ INSERT PATH HERE ------"

# === Step 1: Get valid MRNs from NIfTI filenames ===
cohort_mrns = {f.split('_')[0] for f in os.listdir(nii_folder) if f.endswith('.nii.gz')}

# === Step 2: Load Excel to get only the selected series folders ===
series_df = pd.read_excel(series_excel, dtype=str)
series_df = series_df[series_df['MRN'].isin(cohort_mrns)][['MRN', 'Preop_Series', 'Postop_Series']].dropna()

# === Step 3: Scan for metadata from only the chosen series folders ===
def extract_metadata_once(series_root):
    """Extract metadata from the first valid DICOM file found in a folder."""
    for root, _, files in os.walk(series_root):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                dcm = pydicom.dcmread(file_path, stop_before_pixels=True, force=True)
                contrast = (
                    "Yes" if hasattr(dcm, 'ContrastBolusAgent') and dcm.ContrastBolusAgent not in ["", None]
                    else "No"
                )
                return {
                    "Manufacturer": str(getattr(dcm, 'Manufacturer', 'N/A')),
                    "ReconstructionKernel": str(getattr(dcm, 'ConvolutionKernel', 'N/A')),
                    "KVP": float(getattr(dcm, 'KVP', np.nan)),
                    "TubeCurrent": float(getattr(dcm, 'XRayTubeCurrent', np.nan)),
                    "ContrastUsed": contrast
                }
            except:
                continue
    return None

preop_metadata = []
postop_metadata = []

for _, row in series_df.iterrows():
    mrn = row['MRN']
    preop_series = row['Preop_Series']
    postop_series = row['Postop_Series']
    mrn_path = os.path.join(dicom_root, mrn)

    preop_data = None
    postop_data = None

    # Locate and extract from the exact matching series folder
    for root, _, _ in os.walk(mrn_path):
        if preop_series in os.path.normpath(root):
            preop_data = extract_metadata_once(root)
        if postop_series in os.path.normpath(root):
            postop_data = extract_metadata_once(root)

    if preop_data:
        preop_metadata.append(preop_data)
    if postop_data:
        postop_metadata.append(postop_data)

# === Step 4: Summary stats
def summarize(metadata):
    df = pd.DataFrame(metadata)

    def dist(col):
        counts = df[col].value_counts(dropna=False)
        perc = counts / len(df) * 100
        return pd.DataFrame({'Count': counts, 'Percentage': perc.round(2)})

    def numeric(col):
        values = df[col].dropna()
        return {
            'Min': float(values.min()),
            'Max': float(values.max()),
            'Median': float(values.median()),
            'IQR': (float(values.quantile(0.25)), float(values.quantile(0.75)))
        } if not values.empty else {}

    return {
        'Manufacturer Distribution': dist('Manufacturer'),
        'Kernel Distribution': dist('ReconstructionKernel'),
        'KVP Stats': numeric('KVP'),
        'TubeCurrent Stats': numeric('TubeCurrent'),
        'Contrast Usage': dist('ContrastUsed')
    }

# === Step 5: Print results
preop_summary = summarize(preop_metadata)
postop_summary = summarize(postop_metadata)

print("\n=== PREOPERATIVE CT ACQUISITION SUMMARY ===")
print("Manufacturers:\n", preop_summary['Manufacturer Distribution'])
print("\nReconstruction Kernels:\n", preop_summary['Kernel Distribution'])
print("\nKVP:", preop_summary['KVP Stats'])
print("Tube Current:", preop_summary['TubeCurrent Stats'])
print("\nContrast Usage:\n", preop_summary['Contrast Usage'])  # <-- ADD THIS LINE

print("\n=== POSTOPERATIVE CT ACQUISITION SUMMARY ===")
print("Manufacturers:\n", postop_summary['Manufacturer Distribution'])
print("\nReconstruction Kernels:\n", postop_summary['Kernel Distribution'])
print("\nKVP:", postop_summary['KVP Stats'])
print("Tube Current:", postop_summary['TubeCurrent Stats'])
print("\nContrast Usage:\n", postop_summary['Contrast Usage'])  # <-- AND THIS LINE


## Counting manufacturer metadata

In [ ]:
# === Paths ===
excel_path = r"------ INSERT PATH HERE ------"
dicom_base = r"------ INSERT PATH HERE ------"

# === Load Excel File ===
df = pd.read_excel(excel_path)

# === Helper to find scanner type from series folder ===
def find_scanner_type(series_name, mrn):
    base_path = os.path.join(dicom_base, str(mrn), "DICOM")
    if not os.path.exists(base_path):
        return None

    for root, dirs, files in os.walk(base_path):
        if os.path.basename(root) == str(series_name):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    dcm = pydicom.dcmread(file_path, stop_before_pixels=True, force=True)
                    return dcm.get("Manufacturer", "Unknown")
                except Exception:
                    continue
    return None

# === Process all rows ===
preop_scanners = []
postop_scanners = []

for _, row in df.iterrows():
    mrn = row["MRN"]
    preop_series = row["Preop_Series"]
    postop_series = row["Postop_Series"]

    preop_scanners.append(find_scanner_type(preop_series, mrn))
    postop_scanners.append(find_scanner_type(postop_series, mrn))

# === Add to DataFrame ===
df["Preop_Scanner"] = preop_scanners
df["Postop_Scanner"] = postop_scanners

# === Compute counts and percentages ===
preop_counts = df["Preop_Scanner"].value_counts(normalize=True) * 100
postop_counts = df["Postop_Scanner"].value_counts(normalize=True) * 100

# === Print percentages ===
print("\n📊 Preop Scanner Manufacturer Distribution:")
print(preop_counts.round(2).to_string())

print("\n📊 Postop Scanner Manufacturer Distribution:")
print(postop_counts.round(2).to_string())


## Sensitivity analysis - Dice scores for 2.5mm or more slice thickness scans

In [ ]:
# === Paths ===
preop_pred_dir = r"------ INSERT PATH HERE ------"
postop_pred_dir = r"------ INSERT PATH HERE ------"
preop_gt_dir = r"------ INSERT PATH HERE ------"
postop_gt_dir = r"------ INSERT PATH HERE ------"
slicefile = r"------ INSERT PATH HERE ------"

# === Load slice thickness data ===
df_slices = pd.read_excel(slicefile)
df_slices['MRN'] = df_slices['MRN'].astype(str)

# === Dice calculation ===
def dice(pred, gt):
    intersection = np.sum(pred & gt)
    total = np.sum(pred) + np.sum(gt)
    if total == 0:
        return 1.0
    return 2.0 * intersection / total

def get_mrn_from_filename(fname):
    return ''.join(filter(str.isdigit, fname.split('.')[0]))

def compute_dice_scores(pred_dir, gt_dir, slice_col, labels, label_map):
    scores = {l: [] for l in labels}
    scores['mean'] = []
    per_scan = []  # To store per-scan Dice values

    for fname in os.listdir(pred_dir):
        if not fname.endswith(".nii.gz"):
            continue
        mrn = get_mrn_from_filename(fname)
        row = df_slices[df_slices['MRN'] == mrn]
        if row.empty:
            continue
        thickness = row[slice_col].values[0]
        if pd.isna(thickness) or thickness < 2.5:
            continue

        pred_path = os.path.join(pred_dir, fname)
        gt_path = os.path.join(gt_dir, fname.replace("_postprocessed", "").replace("_final", ""))
        if not os.path.exists(gt_path):
            continue

        pred = nib.load(pred_path).get_fdata().astype(np.uint8)
        gt = nib.load(gt_path).get_fdata().astype(np.uint8)

        per_label_dice = {}
        for label_id in labels:
            label_name = label_map[label_id]
            d = dice(pred == label_id, gt == label_id)
            scores[label_id].append(d)
            per_label_dice[label_name] = d
        mean_d = np.mean(list(per_label_dice.values()))
        scores['mean'].append(mean_d)
        per_scan.append((mrn, per_label_dice, mean_d))
    return scores, per_scan

def summarize(scores, per_scan, label_names, title):
    print(f"\n📊 {title} (Slice Thickness ≥ 2.5 mm):\n")

    for mrn, scan_scores, mean_d in per_scan:
        label_str = ', '.join([f"{k}: {v:.3f}" for k, v in scan_scores.items()])
        print(f"🧪 MRN {mrn} → {label_str} | Mean: {mean_d:.3f}")

    print("\n📈 Summary:")
    for label in label_names:
        arr = np.array(scores[label])
        mean = np.nanmean(arr)
        std = np.nanstd(arr)
        print(f"  {label}: {mean:.3f} ± {std:.3f}")
    mean_overall = np.nanmean(scores['mean'])
    std_overall = np.nanstd(scores['mean'])
    print(f"  Overall Mean: {mean_overall:.3f} ± {std_overall:.3f}")

# === Run
preop_labels = ['RUL', 'RML', 'RLL', 'LUL', 'LLL']
preop_map = {1: 'RUL', 2: 'RML', 3: 'RLL', 4: 'LUL', 5: 'LLL'}
postop_labels = ['RML', 'RLL', 'LLL']
postop_map = {1: 'RML', 2: 'RLL', 3: 'LLL'}

label_ids_preop = list(preop_map.keys())
label_ids_postop = list(postop_map.keys())

# Compute
preop_scores, preop_per_scan = compute_dice_scores(preop_pred_dir, preop_gt_dir, 'Preop_SliceThickness', label_ids_preop, preop_map)
postop_scores, postop_per_scan = compute_dice_scores(postop_pred_dir, postop_gt_dir, 'Postop_SliceThickness', label_ids_postop, postop_map)

# Rename keys for printing
preop_scores_named = {preop_map[k]: v for k, v in preop_scores.items() if k != 'mean'}
preop_scores_named['mean'] = preop_scores['mean']

postop_scores_named = {postop_map[k]: v for k, v in postop_scores.items() if k != 'mean'}
postop_scores_named['mean'] = postop_scores['mean']

# Print
summarize(preop_scores_named, preop_per_scan, preop_labels, "Preop Dice Scores")
summarize(postop_scores_named, postop_per_scan, postop_labels, "Postop Dice Scores")


## Atelectasis grading and plotting using different metrics

In [ ]:
# === Load data ===
file_path = r"------ INSERT PATH HERE ------"
df = pd.read_excel(file_path)

# === Variables ===
grade_cols = [
    "CT_Atelectasis_6m",  # top row
    "CT_Atelectasis_6m_0_12_34"   # bottom row
]
metrics = ["Delta_RML", "Delta_RML_RL", "Delta_RML_TL", "Delta_RLL_RL", "Delta_RLL_TL"]
titles = ["ΔRML", "ΔRML/RL", "ΔRML/TL", "ΔRLL/RL", "ΔRLL/TL"]

# === Plot setup ===
sns.set(style="whitegrid")
fig, axes = plt.subplots(2, 5, figsize=(24, 10), sharey=False)

for row_idx, grade_col in enumerate(grade_cols):
    for col_idx, (metric, title) in enumerate(zip(metrics, titles)):
        ax = axes[row_idx, col_idx]
        sns.boxplot(x=grade_col, y=metric, data=df, ax=ax, showfliers=False)
        sns.stripplot(x=grade_col, y=metric, data=df, ax=ax, color='black', size=4, jitter=True, alpha=0.7)
        
        if row_idx == 0:
            ax.set_title(title, fontsize=13)
        ax.set_xlabel("Atelectasis Grade", fontsize=11)
        ax.set_ylabel(metric if col_idx == 0 else "", fontsize=11)

        # Annotate p-values between adjacent grades
        unique_grades = sorted(df[grade_col].dropna().unique())
        num_tests = len(unique_grades) - 1
        alpha_bonf = 0.05 / num_tests if num_tests > 0 else 1

        for j in range(num_tests):
            g1, g2 = unique_grades[j], unique_grades[j + 1]
            vals1 = df[df[grade_col] == g1][metric].dropna()
            vals2 = df[df[grade_col] == g2][metric].dropna()
            if len(vals1) > 1 and len(vals2) > 1:
                stat, p = mannwhitneyu(vals1, vals2, alternative='two-sided')
                y_max = df[metric].dropna().max()
                y = y_max + 0.05 * (j + 1) * y_max
                ax.plot([j, j + 1], [y, y], color='black')
                if p < 0.001:
                    p_text = "p < 0.001"
                else:
                    p_text = f"p={p:.3f}"
                if p < alpha_bonf:
                    p_text += " *"
                ax.text((j + j + 1) / 2, y, p_text, ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


## Sensitivity analysis - Removing +RML wedge resection 

In [ ]:
# === Load data ===
file_path = r"------ INSERT PATH HERE ------"
df = pd.read_excel(file_path)

# === Exclude rows where RML_w == "w" ===
df = df[df["RML_w"] != "w"]

# === Variables ===
grade_cols = [
    "CT_Atelectasis_6m",  # top row
    "CT_Atelectasis_6m_0_12_34"   # bottom row
]
metrics = ["Delta_RML", "Delta_RML_RL", "Delta_RML_TL", "Delta_RLL_RL", "Delta_RLL_TL"]
titles = ["ΔRML", "ΔRML RL", "ΔRML TL", "ΔRLL RL", "ΔRLL TL"]

# === Plot setup ===
sns.set(style="whitegrid")
fig, axes = plt.subplots(2, 5, figsize=(24, 10), sharey=False)

for row_idx, grade_col in enumerate(grade_cols):
    for col_idx, (metric, title) in enumerate(zip(metrics, titles)):
        ax = axes[row_idx, col_idx]
        sns.boxplot(x=grade_col, y=metric, data=df, ax=ax, showfliers=False)
        sns.stripplot(x=grade_col, y=metric, data=df, ax=ax, color='black', size=4, jitter=True, alpha=0.7)
        
        if row_idx == 0:
            ax.set_title(title, fontsize=13)
        ax.set_xlabel("Atelectasis Grade", fontsize=11)
        ax.set_ylabel(metric if col_idx == 0 else "", fontsize=11)

        # Annotate p-values between adjacent grades
        unique_grades = sorted(df[grade_col].dropna().unique())
        num_tests = len(unique_grades) - 1
        alpha_bonf = 0.05 / num_tests if num_tests > 0 else 1

        for j in range(num_tests):
            g1, g2 = unique_grades[j], unique_grades[j + 1]
            vals1 = df[df[grade_col] == g1][metric].dropna()
            vals2 = df[df[grade_col] == g2][metric].dropna()
            if len(vals1) > 1 and len(vals2) > 1:
                stat, p = mannwhitneyu(vals1, vals2, alternative='two-sided')
                y_max = df[metric].dropna().max()
                y = y_max + 0.05 * (j + 1) * y_max
                ax.plot([j, j + 1], [y, y], color='black')
                if p < 0.001:
                    p_text = "p < 0.001"
                else:
                    p_text = f"p={p:.3f}"
                if p < alpha_bonf:
                    p_text += " *"
                ax.text((j + j + 1) / 2, y, p_text, ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


## Getting distribution plots for the HU values for two training datasets

In [ ]:
# === Paths ===
img_dir = r"------ INSERT PATH HERE ------"
mask_dir = r"------ INSERT PATH HERE ------"

# === Initialization ===
label_range = [1, 2, 3, 4, 5]
data = []

# === Loop over CTs ===
for fname in os.listdir(img_dir):
    if not fname.endswith("_0000.nii.gz"):
        continue

    img_path = os.path.join(img_dir, fname)
    base_id = fname.replace("_0000.nii.gz", "")
    dataset_type = "LLS" if base_id.startswith("LLS") else "MRN"

    mask_fname = base_id + ".nii.gz"
    mask_path = os.path.join(mask_dir, mask_fname)
    if not os.path.exists(mask_path):
        print(f"Missing mask for: {base_id}")
        continue

    img = nib.load(img_path).get_fdata()
    mask = nib.load(mask_path).get_fdata()

    for label in label_range:
        hu_vals = img[mask == label].flatten()
        if len(hu_vals) == 0:
            continue
        sampled_vals = np.random.choice(hu_vals, size=min(5000, len(hu_vals)), replace=False)
        data.extend([(v, dataset_type) for v in sampled_vals])

# === Create DataFrame ===
df = pd.DataFrame(data, columns=["HU", "Dataset"])
df["Dataset"] = df["Dataset"].replace({
    "LLS": "Ottawa chest CTs",
    "MRN": "Stanford chest CTs"
})

# === Manually ensure both categories exist in the plot (even with dummy data if needed) ===
required_datasets = ["Ottawa chest CTs", "Stanford chest CTs"]
for req in required_datasets:
    if req not in df["Dataset"].values:
        df = pd.concat([df, pd.DataFrame({"HU": [-2000], "Dataset": [req]})], ignore_index=True)

# === Plot ===
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x="HU", hue="Dataset", fill=True, common_norm=False, alpha=0.5)

plt.title("Distribution of HU Values Across Dataset Types (All Lobes Combined)")
plt.xlabel("HU Value")
plt.ylabel("Density")
plt.legend(title="Dataset Type")
plt.xlim(-1100, 500)
plt.tight_layout()
plt.show()